In [1]:
import vitaldb
import pyvital.arr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy import signal
import scipy as sp
import pyvital.filters.abp_ppv as f
import matplotlib.pyplot as plt
import statistics
import random
# df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
# df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information
# df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # laboratory results
import numpy as np
import matplotlib.pylab as plt
import padasip as pa
from scipy.signal import find_peaks
from tsfresh.feature_extraction.feature_calculators import time_reversal_asymmetry_statistic
import math
import scipy
import scipy.integrate
from statsmodels.tsa.seasonal import STL
import pyvital
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh import extract_features
from statsmodels.tsa.seasonal import STL
import seaborn as sns
import pandas as pd
import pandas
import smogn
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import seaborn as sns
from skrvm import RVR
#import catboost as cb
from sklearn.tree import DecisionTreeRegressor
import xgboost
from xgboost import XGBRegressor
from scipy import signal
import biosppy
import neurokit2 as nk
import random

Download tracks, cases, and labs

In [3]:
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information
df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # laboratory results

caseids_SNUADC_ART_Vigilance = list(
    set(df_trks.loc[df_trks['tname'] == 'Vigilance/CO', 'caseid'] ) &
    set(df_trks.loc[df_trks['tname'] == 'Vigilance/SV', 'caseid'] ) &
    set(df_trks.loc[df_trks['tname'] == 'SNUADC/ART', 'caseid']) & 
    set(df_trks.loc[df_trks['tname'] == 'SNUADC/ECG_V5', 'caseid']) & 
    set(df_cases.loc[df_cases['age'] > 18, 'caseid'])   
)

print('Total {} cases found'.format(len(caseids_SNUADC_ART_Vigilance)))
np.random.shuffle(caseids_SNUADC_ART_Vigilance)  # shuffle caseids

Total 54 cases found


Define variables to store the data

In [4]:

data = []
ABP = []
ABP_current = []
indexCO = []
fullData = []
CO = []
CO_current = []
check = 1
patient_height = []
patient_weight = []
patient_age = []
patient_sex = []
patient_bmi = []
HR = []
HR_signal = []
systolic_peaks = []
onset_peaks = []
notches = []
nCardiac = 2
art_smoothed = []
data = []
num_of_cases = 12
num_of_samples = 3
features = []
counter = -1
MAP = []
length = []
variance = []
sum_of_derivative = []
SD = []
sys_pressure = []
onset_pressure = []
abnormals = []
abn = 0
nac_abn = []   
segment = []
patient_ppf = []
ID = []


Download the data

In [5]:
for cases in range(0,len(caseids_SNUADC_ART_Vigilance)):
  
  nc = cases
  valsEV1000 = np.array(vitaldb.load_case(caseids_SNUADC_ART_Vigilance[nc], ['SNUADC/ART','Vigilance/CO','Vigilance/SV','Vigilance/HR_AVG','SNUADC/ECG_V5'], 1/100)) # load cases that have ABP and CO and downsample it
  patient_age.append(df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'age'])
  patient_weight.append(df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'weight'])
  patient_sex.append(df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'sex'])
  patient_height.append(df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'height'])
  patient_bmi.append(df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'bmi'])
  ID.append(caseids_SNUADC_ART_Vigilance[cases])

  indexCO = np.array(np.where(~np.isnan(valsEV1000[:,2])))
  print(cases)
  loop = 300 # replace this by the number of samplse you want to upload. If you wish to download everything use loop = len(indexCO)
  if len(indexCO[0]) < 300:
    print('case does have enough data')
    loop = len(indexCO[0])-round(len(indexCO[0])/700)
                   
  for ABP_sample in range(0,loop):
    nac = ABP_sample
    #collect art samples
    ABP_current = np.array(valsEV1000[indexCO[0,nac]-1500:indexCO[0,nac],0]) # collect "sample" from ABP starting from the previous time CO is estimated up till the current estimation
    #HR = np.array(valsEV1000[indexCO[0,nac]-1000:indexCO[0,nac],3])  
    # collect hr samples
    valid = True
    validi = True
    if (ABP_current > 200).any():
      valid = False
      #print(1)
    elif(ABP_current < 30).any():
      valid = False
      #print(2)
    elif np.max(ABP_current) - np.min(ABP_current) < 30:
      valid = False
      #print(3)
    elif (np.abs(np.diff(ABP_current)) > 30).any():  # abrupt change -> noise
      valid = False
      #print(4)
    elif np.isnan(ABP_current).any():
      valid = False
      #print(5)
    if valid == False:
      validi == False
      nac_abn.append(nac)
      #print("something went wrong")
    cnt = 0
    while validi == False:
        cnt = cnt + 1
        if cnt == 100:
           continue
        nac = nac + 1
        if nac >= len(indexCO[0]):
           validi = True
           continue
        ABP_current = np.array(valsEV1000[indexCO[0,nac]-1500:indexCO[0,nac],0]) # collect "sample" from ABP starting from the previous time CO is estimated up till the current estimation
        #HR = np.array(valsEV1000[indexCO[0,nac]-1000:indexCO[0,nac],3])  
        # collect hr samples
        validi = True
        if (ABP_current > 200).any():
            validi = False
        elif(ABP_current < 30).any():
            validi = False
        elif np.max(ABP_current) - np.min(ABP_current) < 30:
            validi = False
        elif (np.abs(np.diff(ABP_current)) > 30).any():  # abrupt change -> noise
            validi = False
        # elif np.isnan(HR).any():
        #     valid = False
        elif np.isnan(ABP_current).any():   
            validi = False  
    if validi == False:
       print("something went wrong")
       continue
    
    HR_current = np.array(valsEV1000[indexCO[0,nac],3])
        # collect co samples
    CO_current = np.array(valsEV1000[indexCO[0,nac],1])
    SV_current = np.array(valsEV1000[indexCO[0,nac],2])
    data = []
    # print(ID[cases])
    data = np.append(data, caseids_SNUADC_ART_Vigilance[cases])
    data = np.append(data, HR_current)
    data = np.append(data, CO_current)
    data = np.append(data, SV_current)
    data = np.append(data, patient_weight[cases])
    data = np.append(data, patient_age[cases])
    abn = 0
    if df_cases.at[caseids_SNUADC_ART_Vigilance[nc],'sex'] == 'M':
      data = np.append(data, 1) # if famle set 1
    else:
      data = np.append(data, 2) # if male set 2
    data = np.append(data, patient_height[cases])
    data = np.append(data, patient_bmi[cases])
    data = np.append(data, ABP_current)
    data = np.append(data, HR)
    if ABP_sample == 0 and cases == 0:
      segment = data
    else:
      segment = np.vstack([segment, data])
        # append the values of art,hr,en co
    # filter the signal
df = pd.DataFrame(segment)

0
1
2
3
